**Important**: This notebook only works together with the oggm dev branch at the moment and requires SALib to be installed.

In [ ]:
#!pip install SALib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from oggm import utils, workflow, tasks, cfg
from oggm.core.massbalance import MonthlyTIModel

import SALib.sample.sobol as sampler
import SALib.analyze.sobol as analyser

# Set up a gdir to play with

In [ ]:
cfg.initialize(logging_level="WARNING")

working_dir = 'working_dir_mcs'
utils.mkdir(working_dir)
cfg.PATHS["working_dir"] = working_dir

In [ ]:
rgi_ids = ["RGI60-06.00377"]
base_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.3/elev_bands/W5E5/'
gdirs = workflow.init_glacier_directories(
        rgi_ids,
        from_prepro_level=3,
        prepro_border=10,
        prepro_rgi_version="62",
        prepro_base_url=base_url,
    )

gdir = gdirs[0]

In [ ]:
# Redo calibration to get the observations values stored in the observations file (prepro gdirs are not created with dev)
workflow.execute_entity_task(tasks.mb_calibration_from_hugonnet_mb,
                             gdirs,
                             informed_threestep=True,  # only available for 'GSWP3_W5E5'
                            );

# Define a workflow to play with

Here you need to define a function which takes all parameters which are included in the uncertainty porpagation. Those parameters include observations (in my example reference mass balance `ref_mb`) and model parameters (in my example `melt_f`, `prcp_fac` and `temp_bias`). The function should conduct the calibration and potentially also a dynamic model run. All results should be stored using the `settings_filesuffix`.

Notes on the new handling of settings and observations:

- **Observations**: In observations, the convention is that the variable name should match the function’s kwarg. For example, `tasks.mb_calibration_from_scalar_mb` has the kwargs `ref_mb`, `ref_mb_period`, and `ref_mb_err`. Therefore, in the observations we use the key `ref_mb`. The actual value is stored under `observations['ref_mb']['value']`, and the other parameters are stored without repeating `ref_mb`. For example, `ref_mb_period` is stored as `observations['ref_mb']['period']`.
- **Settings with a parent file**: A settings file can have a parent. If a parameter is not found in the current file, it will be taken from the parent. By default, the parent is `cfg.PARAMS`. You can change this by setting `parent_filesuffix`. In my example, I use this feature to ensure all parameters are consistent with the default calibration.
- **Calibrated parameters**: After calibration, the parameters are written back into the settings file with the chosen `settings_filesuffix`. This means that after running `tasks.mb_calibration_from_scalar_mb`, you will find the calibrated parameters in the settings under this suffix. In some cases, the initial guess of a parameter may also come from the settings and later be overwritten with the calibrated values. In the future, we might want to add safeguards for this.

In [ ]:
def mcs_mb_calibration(gdir, settings_filesuffix, settings_parent_filesuffix,
                       observations_filesuffix,
                       ref_mb, ref_mb_period, ref_mb_err,
                       prcp_fac, melt_f, temp_bias,
                      ):
    # add observations data to observations file
    gdir.observations_filesuffix = observations_filesuffix
    gdir.observations['ref_mb'] = {'err': ref_mb_err,
                                   'period': ref_mb_period,
                                   'value': ref_mb}

    # create the new settings file with the correct parent_filesuffix
    utils.ModelSettings(gdir, filesuffix=settings_filesuffix,
                        parent_filesuffix=settings_parent_filesuffix)
    gdir.settings_filesuffix = settings_filesuffix
    gdir.settings['use_winter_prcp_fac'] = False  # this is checked in mb_calibration_from_scalar_mb

    # the settings here mimic mb_calibration_from_hugonnet_mb
    workflow.execute_entity_task(tasks.mb_calibration_from_scalar_mb,
                                 gdir,
                                 settings_filesuffix=settings_filesuffix,
                                 observations_filesuffix=observations_filesuffix,
                                 overwrite_gdir=True,
                                 calibrate_param1='prcp_fac',
                                 calibrate_param2='melt_f',
                                 calibrate_param3='temp_bias',
                                 prcp_fac=prcp_fac,
                                 prcp_fac_min=prcp_fac * 0.8,
                                 prcp_fac_max=prcp_fac * 1.2,
                                 melt_f=melt_f,
                                 temp_bias=temp_bias,
                                 mb_model_class=MonthlyTIModel,
                                )

# Create sample of input parameters

Here we define the distribution of the parameters we consider (observations and model parameters). This example here was just a quick setup for testing the principle workflow and need to be refined carefully depending on the application and the used type of observation. 

Here an overview which kind of distributions can be used and the meaning of 'bounds' depending on the selected distribution: https://salib.readthedocs.io/en/latest/user_guide/advanced.html#generating-alternate-distributions

## define parameter distributions

I included here a small in between step for defining the parameters in a more readable way and only at the end convert it into the format which is expected by the sampler.

In [ ]:
# to have a better overview I provide the parameter distribution in a more readable way first
parameter_distribution = {}

# add geodetic mb observation
gdir.observations_filesuffix = ''  # just to be sure to use the correct observations
ref_mb = gdir.observations['ref_mb']
parameter_distribution['ref_mb'] = {
    'bounds': [ref_mb['value'] - ref_mb['err'],  # this need to be checked again if we should use +/-err/2
               ref_mb['value'] + ref_mb['err']],
    'dists': 'unif'  # need to be checked
}

# add mb parameter distributions (they are made up and need to be decided somehow)
gdir.settings_filesuffix = ''
parameter_distribution['prcp_fac'] = {
    'bounds': [max(gdir.settings['prcp_fac'] * 0.5, gdir.settings['prcp_fac_min']),
               min(gdir.settings['prcp_fac'] * 1.5, gdir.settings['prcp_fac_max'])
              ],
    'dists': 'unif'
}
parameter_distribution['melt_f'] = {
    'bounds': [max(gdir.settings['melt_f'] - 1, gdir.settings['melt_f_min']),
               min(gdir.settings['melt_f'] + 1, gdir.settings['melt_f_max'])],
    'dists': 'unif'
}
parameter_distribution['temp_bias'] = {
    'bounds': [max(gdir.settings['temp_bias'] - 0.5, gdir.settings['temp_bias_min']),
               min(gdir.settings['temp_bias'] + 0.5, gdir.settings['temp_bias_max'])],
    'dists': 'unif'
}

# now convert to format which is expected by SALib
num_vars = 0
problem = {
    'names': [],
    'bounds': [],
    'dists': [],
}

for key in parameter_distribution:
    num_vars += 1
    problem['names'].append(key)
    problem['bounds'].append(parameter_distribution[key]['bounds'])
    problem['dists'].append(parameter_distribution[key]['dists'])
problem['num_vars'] = num_vars

In [ ]:
problem

## create actual samples

Creating the actual sample is quite easy again. For more information on the sampling method you can read up the references provided in the sampler docstring `sampler.sample?`.

In [ ]:
# sampler.sample?

In [ ]:
# the total number of samples created is 
# - including second order interconnections: N * (2 * num_vars + 2)
# - or only look at uncorrelated sensitivity with calc_second_order=False: N * (num_vars + 2)
N = 2**5  # if using second order this needs to be a power of 2
param_values = sampler.sample(problem, N, calc_second_order=True)

In [ ]:
param_values.shape

# execute a calibration for each parameter setting

Here I loop through all samples and execute my function I defined above. For this we probably could utilize multiprocessing in the future (one option would be to make our function an entity_task and use the execute_entity_task logic).

In [ ]:
# for the reference mass balance we provide the same error and mb_period as the original observation
gdir.observations_filesuffix = ''
ref_mb = gdir.observations['ref_mb']

for i, param_val in enumerate(param_values):
    print(f"{i + 1}/{param_values.shape[0]}")
    mcs_mb_calibration(gdir,
                       settings_filesuffix=f'_{i}',
                       settings_parent_filesuffix='',
                       observations_filesuffix=f'_{i}',
                       ref_mb_period=ref_mb['period'],
                       ref_mb_err=ref_mb['err'],
                       ref_mb=param_val[problem['names'].index('ref_mb')],
                       prcp_fac=param_val[problem['names'].index('prcp_fac')],
                       melt_f=param_val[problem['names'].index('melt_f')],
                       temp_bias=param_val[problem['names'].index('temp_bias')],
                      )

# Look at results

## resulting distribution of mb parameters

Here just look at the resulting mass balance parameters after calibration. We initialized all with a uniform distribution.

In [ ]:
# default run parameters
gdir.settings_filesuffix = ''
prcp_fac_default = gdir.settings['prcp_fac']
melt_f_default = gdir.settings['melt_f']
temp_bias_default = gdir.settings['temp_bias']

prcp_fac_all = []
melt_f_all  =[]
temp_bias_all = []

for i in range(param_values.shape[0]):
    gdir.settings_filesuffix = f'_{i}'
    prcp_fac_all.append(gdir.settings['prcp_fac'])
    melt_f_all.append(gdir.settings['melt_f'])
    temp_bias_all.append(gdir.settings['temp_bias'])

In [ ]:
def plt_parameter_distribution(para_all, para_default, title, bins=20):
    plt.hist(para_all, bins=bins, color='C0')
    plt.axvline(para_default, c='k', label='control run')
    plt.axvline(np.median(para_all), c='C1', label='median from sample')
    plt.axvline(np.mean(para_all), c='C1', ls='--', label='mean from sample')
    plt.title(title)
    plt.legend()
    plt.show()

plt_parameter_distribution(prcp_fac_all,
                           prcp_fac_default,
                           'prcp_fac',
                           bins=20)

plt_parameter_distribution(melt_f_all,
                           melt_f_default,
                           'melt_f',
                           bins=20)

plt_parameter_distribution(temp_bias_all,
                           temp_bias_default,
                           'temp_bias',
                           bins=20)


## pick one year and look at distribution of resulting mb values

Here we visualize the uncertainty for the specific mass balance for the period 1980 to 2020 as defined by the MCS sample.

In [ ]:
fls = gdir.read_pickle('inversion_flowlines')
years = np.arange(1980, 2020)

# get control run (= current default OGGM calibration)
mb_mod = MonthlyTIModel(gdir, settings_filesuffix='')
specific_mb_default = mb_mod.get_specific_mb(fls=fls, year=years)

# now get values from MCS sample
specific_mb_all = []
for i in range(param_values.shape[0]):
    mb_mod = MonthlyTIModel(gdir, settings_filesuffix=f'_{i}')
    specific_mb_all.append(mb_mod.get_specific_mb(fls=fls, year=years))
specific_mb_all = np.array(specific_mb_all)

In [ ]:
plt.fill_between(years, np.min(specific_mb_all, axis=0), np.max(specific_mb_all, axis=0),
                 color='lightgray', label=f'range MCS sample')

plt.plot(years, specific_mb_default, '-k', label='control run (default)')
plt.plot(years, np.median(specific_mb_all, axis=0), 'C1--', label='median MCS sample')
plt.plot(years, np.mean(specific_mb_all, axis=0), 'C2:', label='mean MCS sample')
plt.title('Example of the uncertainties generated with MCS')
plt.ylabel('Specific MB (mm w.e.)')
plt.legend()
plt.show()

# Bonus: Get sensitivities (not needed at this stage of DTC-Glaciers)

This is not needed at the moment. But the idea is that you also can analyze how much of the variance of a result is comming from the observations and how much from the model parameters. This could become interesting in a next phase of DTC-Glaciers.

In [ ]:
# compare how much of the variance for one year is comming from the observation and how much from the model
yr = 2000

# now get values from MCS runs
specific_mb_yr = []
for i in range(param_values.shape[0]):
    mb_mod = MonthlyTIModel(gdir, settings_filesuffix=f'_{i}')
    specific_mb_yr.append(mb_mod.get_specific_mb(fls=fls, year=yr))
specific_mb_yr = np.array(specific_mb_yr)

In [ ]:
Si = analyser.analyze(problem, specific_mb_yr, print_to_console=True)

How to read this (from my amateur point of few, without digging into the references):

In the first order sensitivities S1 we see the largest portion of the sensitivity of the 2000 specific mass balance is comming from the precipitation factor followed by the melt_factor.

To get a more indepth understanding have a look at the references provided in the docstring `analyser.analyze?`.

In [ ]:
# analyser.analyze?